In [1]:
################################################################
# Ordinary Kriging
# Written by Shen Xianwen
# 2020.01.07 ver.1.0
# Verification:
# - Windows10 Home x64
# - Python     3.7.3
# - numpy      1.15.4
# - matplotlib 3.0.3
################################################################

import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import minimize
import pandas as pd

def genCovMat(x, y, dcor, sigma):
    dmat = distance(x, y, x[:, np.newaxis], y[:, np.newaxis]) # distance matrix
    tmp = np.log(2.0) / dcor # np.log(2.0)/dcor

    return sigma * sigma * np.exp(-dmat * tmp)


def genSemivar(data, d_max, num):
    def genCombinations(arr):
      r, c = np.triu_indices(len(arr), 1) #Return the indices for the upper-triangle of an (n, m) array.
      return np.stack((arr[r], arr[c]), 1)

    d_semivar = np.linspace(0.0, d_max, num)
    SPAN = d_semivar[1] - d_semivar[0]

    indx = genCombinations(np.arange(8))
    d = distance(data[indx[:, 0], 0], data[indx[:, 0], 1], data[indx[:, 1], 0], data[indx[:, 1], 1])
    indx = indx[d<=d_max]
    d = d[d <= d_max]
    semivar = (data[indx[:, 0], 2] - data[indx[:, 1], 2])**2

    semivar_avg = np.empty(num)
    for i in range(num):
        d1 = d_semivar[i] - 0.5*SPAN
        d2 = d_semivar[i] + 0.5*SPAN
        indx_tmp = (d1 < d) * (d <= d2) #index within calculation span
        semivar_tr = semivar[indx_tmp]
        semivar_avg[i] = semivar_tr.mean()

    return d_semivar[np.isnan(semivar_avg) == False], 0.5 * semivar_avg[np.isnan(semivar_avg) == False]


def semivar_exp(d, nug, sill, ran):
  return np.abs(nug) + np.abs(sill) * (1.0-np.exp(-d/(np.abs(ran))))


def semivarFitting(d, data):
    def objFunc(x):
        theorem = semivar_exp(d, x[0], x[1], x[2])
        return ((data-theorem)**2).sum()

    x0 = np.random.uniform(0.0, 1.0, 3)
    res = minimize(objFunc, x0, method='nelder-mead')
    for i in range(5):
        x0 = np.random.uniform(0.0, 1.0, 3)
        res_tmp = minimize(objFunc, x0, method='nelder-mead')
        if res.fun > res_tmp.fun:
            res = res_tmp
    return np.abs(res.x)

def ordinaryKriging(mat, x_vec, y_vec, z_vec, x_rx, y_rx, nug, sill, ran):
    vec = np.ones(len(z_vec)+1, dtype=np.float)

    d_vec = distance(x_vec, y_vec, x_rx, y_rx)
    vec[:len(z_vec)] = semivar_exp(d_vec, nug, sill, ran)
    weight = np.linalg.solve(mat, vec)
    est = (z_vec * weight[:len(z_vec)]).sum()

    return est

def distance(x1, y1, x2, y2):
    return np.sqrt((x1-x2)**2 + (y1-y2)**2)

def genMat(x_vec, y_vec, z_vec, nug, sill, ran):
    mat = distance(x_vec, y_vec, x_vec[:, np.newaxis], y_vec[:, np.newaxis])
    mat = semivar_exp(mat, nug, sill, ran)
    mat = np.vstack((mat, np.ones(len(z_vec))))
    mat = np.hstack((mat, np.ones([len(z_vec)+1, 1])))
    mat[len(z_vec)][len(z_vec)] = 0.0

    return mat

if __name__=="__main__":
    
    D_MAX = 500.0 #maximum distance in semivariogram modeling
    N_SEMIVAR = 8 #number of points for averaging empirical semivariograms
    
    known_data = pd.read_excel(r'C:\Users\Trine\Desktop\homework\LatitudeLogitude_8Stations_Known.xls')
    pre_data = pd.read_excel(r'C:\Users\Trine\Desktop\homework\LatitudeLogitude_6Stations_ToBeEstimated.xls')
    rain_data = pd.read_excel(r'C:\Users\Trine\Desktop\homework\Rain_15year_8Stations_Known.xls')
    x = known_data.loc[:, 'latitude'].values
    y = known_data.loc[:, 'logitude'].values
    z = rain_data.mean(0)

    data = np.vstack([x, y, z]).T
    d_sv, sv = genSemivar(data, D_MAX, N_SEMIVAR)
    param = semivarFitting(d_sv, sv)

    x_valid = pre_data.loc[:, 'latitude'].values
    y_valid =  pre_data.loc[:, 'logitude'].values
    X, Y = np.meshgrid(x_valid, y_valid)
    z_map = np.zeros([len(x_valid), len(y_valid)])

    mat = genMat(x, y, z, param[0], param[1], param[2])
    for i in range(len(y_valid)):
        for j in range(len(x_valid)):
            z_map[i][j] = ordinaryKriging(mat, x, y, z, x_valid[j], y_valid[i], param[0], param[1], param[2])
            
    table_d = dict(zip(pre_data.loc[:, 'station name'], z_map[X[0,:] == pre_data.loc[:, 'latitude'], 
                                                          Y[:,0] == pre_data.loc[:, 'logitude']]))
    table = pd.DataFrame(table_d, index=list([1]))

C:\Users\Trine\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: Mean of empty slice.
C:\Users\Trine\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [3]:
table = pd.DataFrame(data = z_map[X[0,:] == pre_data.loc[:, 'latitude'], Y[:,0] == pre_data.loc[:, 'logitude']],
                     index = pre_data.loc[:, 'station name'])

In [26]:
table_d = dict(zip(pre_data.loc[:, 'station name'], z_map[X[0,:] == pre_data.loc[:, 'latitude'], 
                                                          Y[:,0] == pre_data.loc[:, 'logitude']]))
table = pd.DataFrame(table_d, index=list([1]))
table

,byers,estes,golden,green9se,lakegeor,morrison
1,8887.065614,8043.672953,8128.558735,8569.518194,8309.838688,8156.876628


In [33]:
import numpy.matlib as mlt
def Fint(x0, f0, xint, p = 2, rad = float("inf"), L = 2):
    n = len(x0[0])
    m = len(x0)
    q = len(xint[0][0])
    print(xint)
    fint = np.zeros(q)
    
    for i in range(q):
        delta_x = x0 - mlt.repmat(xint[i], n, 1)
        
        dabs_l = np.zeros(len(DeltaX[0]))
        
        for ncol in range(m):
            dabs_l = dabs_l + np.power(abs(delta_x[:,ncol],L))
        d_mat = np.power(dabs_l, 1/L)
        d_mat[d_mat > rad] = float("inf")
        
        W = np.power(d_mat, -p)
        fint[i] = sum(w * F0) / sum(W)
    return fint